In [42]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [43]:
consolidateSalesReports = pd.read_csv('Consolidate Sales Reports.csv', encoding='latin1')


In [44]:
# numListOfBranch = ['401 Co','402 Co','404 Co','405 Co','412 Co','416 Co',
#                    '417 Co','423 Co', '424 Co','425 Co','426 Co','429 Co','444 Co','490 Co']

monthesList = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
listOfBranch = ['ARQOOB SOH','Retailer SOH', 'Final_Cost', 'Month', 'Weeks',
                'Product Lead Time (Days)', 'Last Update']

productDetail = ['Part No', "Description", 'UPC']
header = productDetail.copy()
header.extend(listOfBranch.copy())


In [45]:
supplierForecast = pd.DataFrame(columns = list(header))
for col in supplierForecast.columns:
    supplierForecast[col] = consolidateSalesReports[col]


In [46]:
realMonthes = []
lastMonth = ''
for month in monthesList:
    try:
        supplierForecast[month + '_QTY'] = consolidateSalesReports[month[:3] + '_Total']
        realMonthes.append(month + '_QTY')
        lastMonth = month + '_QTY'
    except:
        continue


In [47]:
QTY_Sold =0
for rm in realMonthes:
    QTY_Sold = QTY_Sold + supplierForecast[rm]
supplierForecast['QTY Sold'] = QTY_Sold

avgPerDay = []
avgPerMonth = []
i = 0
for m in supplierForecast['Month']:
    if supplierForecast.loc[i,lastMonth] == 0:
        if m == 0:
            avgPerDay.append(0)
            avgPerMonth.append(0)
        else:
            avgPerDay.append(np.around(QTY_Sold[i] / (m * 30), 2))
            avgPerMonth.append(np.around(QTY_Sold[i] / m,2))
    else:
        avgPerDay.append(np.around(QTY_Sold[i]/((m*30)+(supplierForecast.loc[i,'Weeks']*7)),2))
        avgPerMonth.append(np.around(QTY_Sold[i] / m,2))
    i += 1
supplierForecast['Avh per Day'],supplierForecast['Avg per Month'] = avgPerDay, avgPerMonth
i = 0
closingStock = []
for soh in supplierForecast['ARQOOB SOH']:
    try:
        int(supplierForecast.loc[i,'ARQOOB SOH'])
    except:
        supplierForecast.loc[i,'ARQOOB SOH'] = 0
    try:
        int(supplierForecast.loc[i,'Retailer SOH'])
    except:
        supplierForecast.loc[i,'Retailer SOH'] = 0
    closingStock.append(supplierForecast.loc[i,'Retailer SOH'] + supplierForecast.loc[i,'ARQOOB SOH'])
    i += 1

supplierForecast['Closing Stock'] = closingStock
i = 0
stockInHand = []
for cl in supplierForecast['Closing Stock']:
    if supplierForecast.loc[i,'Avh per Day'] !=0:
        stockInHand.append(cl/supplierForecast.loc[i,'Avh per Day'])
    else:
        stockInHand.append("Not Sold")
    i += 1
supplierForecast['Days Stock in Hand'] = stockInHand

i = 0
minStock = []
for day in supplierForecast['Days Stock in Hand']:
    try:
        minStock.append(int(supplierForecast.loc[i,'Days Stock in Hand']))
    except:
        minStock.append('-')
    
supplierForecast['Deviation from Minimum Stock Level'] = minStock


supplierForecast['MSL (2 Mont Covering)'] = supplierForecast['Avg per Month'] * 2

stockCover = []
i = 0 
for day in supplierForecast['Days Stock in Hand']:
    try:
        date_1 = supplierForecast.loc[i,'Last Update']

        stockCover.append(str(pd.to_datetime(date_1) + pd.DateOffset(days=int(day)))[:int(10)])
    except:    
        stockCover.append('Not Sold')
    i += 1
supplierForecast['Current Stock Cover upto (Date)'] = stockCover

ArrivalDate = []
i = 0 
for day in supplierForecast['Product Lead Time (Days)']:
    try:
        date_1 = supplierForecast.loc[i,'Last Update']
        newDate = str(pd.to_datetime(date_1) + pd.DateOffset(days=int(day)))[:int(10)]
        ArrivalDate.append(newDate)
    except:    
        ArrivalDate.append('Not Sold')
    i += 1
supplierForecast['Arrival Date'] = ArrivalDate


ArrivalDateETA = []
i = 0 
for day in supplierForecast['Arrival Date']:
    try:
        int(day[0])
        
        month= day[5:7]
        day = day.replace(month + '-',"")
        day = day + '-' + month
        
        coverDay = supplierForecast.loc[i,'Current Stock Cover upto (Date)']
        int(coverDay[0])
        month = coverDay[5:7]
        coverDay = coverDay.replace(coverDay + '-', '')
        coverDay = coverDay + '-' + month
        print(coverDay, day)
        dateList = [day, coverDay]
        ArrivalDateETA.append(max(dateList))
        print(ArrivalDateETA)
    except:    
        ArrivalDateETA.append('Not Sold')
    i += 1
supplierForecast['Arrival Date (ETA)'] = ArrivalDateETA



2019-12-29-12 2019-10-08
['Not Sold', '2019-12-29-12']
2020-05-22-05 2019-10-08
['Not Sold', '2019-12-29-12', '2020-05-22-05']
2019-11-01-11 2019-10-08
['Not Sold', '2019-12-29-12', '2020-05-22-05', '2019-11-01-11']
2019-09-22-09 2019-10-08
['Not Sold', '2019-12-29-12', '2020-05-22-05', '2019-11-01-11', '2019-10-08']
2019-10-06-10 2019-10-08
['Not Sold', '2019-12-29-12', '2020-05-22-05', '2019-11-01-11', '2019-10-08', 'Not Sold', '2019-10-08']
2020-02-12-02 2019-10-08
['Not Sold', '2019-12-29-12', '2020-05-22-05', '2019-11-01-11', '2019-10-08', 'Not Sold', '2019-10-08', 'Not Sold', '2020-02-12-02']
2020-07-24-07 2019-10-08
['Not Sold', '2019-12-29-12', '2020-05-22-05', '2019-11-01-11', '2019-10-08', 'Not Sold', '2019-10-08', 'Not Sold', '2020-02-12-02', 'Not Sold', 'Not Sold', 'Not Sold', 'Not Sold', 'Not Sold', 'Not Sold', 'Not Sold', 'Not Sold', 'Not Sold', 'Not Sold', '2020-07-24-07']
2022-06-27-06 2019-10-08
['Not Sold', '2019-12-29-12', '2020-05-22-05', '2019-11-01-11', '2019-10-0

In [48]:
supplierForecast.to_csv('supplierForecast.csv', index=False)

In [120]:
str(pd.to_datetime(date_1) + pd.DateOffset(days=int(1)))[:int(10)]

'2019-07-28'